In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ignacio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ignacio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
try:
    df = pd.read_csv('../data/comments.csv', index_col='Id', skip_blank_lines=True)
    df = df.dropna()
except Exception as e:
    print(f"Error loading data: {e}")
    exit()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [35]:
len(df)

1000000

In [36]:
def preprocess_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[^A-Za-z\s]', '', text, flags=re.IGNORECASE)
    text = text.lower()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['Processed_Comment'] = df['Comment'].apply(preprocess_text)

#TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000) 
X = vectorizer.fit_transform(df['Comment'])

In [37]:
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)

def get_sentiment_batch(texts):
    try:
        results = sentiment_pipeline(texts, truncation=True, max_length=512, batch_size=32, device=0)
        return [result['label'] for result in results]
    except Exception as e:
        print(f"Error in sentiment analysis: {e}")
        return ["NEUTRAL"] * len(texts)

# Define batch size
batch_size = 100

# Initialize an empty list to store all sentiments
all_sentiments = []

# Create a tqdm progress bar for the entire dataset
with tqdm(total=len(df), desc="Processing comments") as pbar:
    for i in range(0, len(df), batch_size):
        # Get the current batch
        batch_texts = df['Processed_Comment'].iloc[i:i+batch_size].tolist()
        
        # Process the batch
        batch_sentiments = get_sentiment_batch(batch_texts)
        
        # Extend the list of all sentiments
        all_sentiments.extend(batch_sentiments)
        
        # Update the progress bar
        pbar.update(len(batch_texts))

# Assign sentiments to DataFrame
df['Sentiment'] = all_sentiments


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing comments:   0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
# Analyze results
sentiment_counts = df['Sentiment'].value_counts()
print(sentiment_counts)

# Visualize results
plt.figure(figsize=(10,6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values)
plt.title('Distribution of Sentiments')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

# Save results
df.to_csv('../data/comments_with_sentiment.csv')


In [7]:
len(df)

1000019